## Step 0 :- Importing necessary libraries for analysis 

In [1]:
import numpy as np 
import pandas as pd
import os 
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

## Step 1 :- Importing dataset 
<div class="alert alert-block alert-info">
<b>Step 1:</b> Downloading the dataset in form of a pandas dataframe. <br>
<b>Step 2:</b> Selecting features to be used for creating X matrix. <br>
<b>Step 3:</b> Label encoding Y vector<br>
</div>

In [2]:
filepath          = "D:\\kaggle_trials\\voicegender\\voice.csv"
datafile          = pd.read_csv(filepath)
colnames          = datafile.columns
features_needed   = colnames[:-1]
prediction_labels = colnames[-1]
label_enc         = LabelEncoder()
y                 = label_enc.fit_transform(datafile[prediction_labels])
X                 = datafile[features_needed].values

## Step 2:- Performing train test split 
<b>Split ratio used :</b> 67% training data and remaining 33% testing data <br>

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


## Step 3:- Configuration sheet download
<div class="alert alert-block alert-info">
<b>Base estimators:</b> We use Decision Tree Classifier as our base estimator and we will download the necessary parameters related to it. <br>
<b> Main estimator: </b> Our classifier will be Adaboost built upon DecisionTreeClassifier. AdaBoostClassifier has some parameters of its own and we will upload them here as well. 
</div>

In [5]:
configuration_sheet_path = os.getcwd()+'\\configuration_sheet.xlsx'
base_estimator_params = pd.read_excel(configuration_sheet_path,sheet_name='Base_estimator_parameters')
adaboost_params       = pd.read_excel(configuration_sheet_path,sheet_name='Adaboost_parameters')

In [44]:
base_estimator_params.head(3)

,max_depth,min_samples_split,min_samples_leaf
0,1.0,2,1
1,2.0,3,2
2,3.0,4,3


In [45]:
adaboost_params.head(2)

,algorithm,n_estimators,learning_rate
0,SAMME,40.0,0.1
1,SAMME.R,50.0,0.5


## Step 4 :- Instantiating a instance of both the classifiers
<div class="alert alert-block alert-info">
<b>Parameters:</b> There will be some parameters which we won't be tuning at this moment and we will proceed ahead with some constant parameters in the tuning phase in the next step.
</div>

In [6]:
decision_tree_classifier  = DecisionTreeClassifier(splitter = 'best', criterion = 'entropy',random_state = 11)
adaboost_classifier       = AdaBoostClassifier(base_estimator=decision_tree_classifier,random_state=0)

## Step 5:- Parameter tuning
<div class="alert alert-block alert-info">
<b>Parameters to tune: </b> We apply GridSearchCV for tuning 4 parameters of the model and we perform 10 fold cross validations in the tuning process. One parameter(max_depth) is for the base estimator and the remaining ones are for the Adaboost Classifier. 
</div>

In [7]:
parameter_grid = {
                    'base_estimator__max_depth'        : base_estimator_params['max_depth'].dropna().values.tolist(),
#                     'base_estimator__min_samples_split': base_estimator_params['min_samples_split'].dropna().values.tolist(),
#                     'base_estimator__min_samples_leaf' : base_estimator_params['min_samples_leaf'].dropna().values.tolist(),
                    'algorithm'                        : adaboost_params['algorithm'].dropna().values.tolist(),
                    'n_estimators'                     : np.int_(adaboost_params['n_estimators'].dropna().values).tolist(),
                    'learning_rate'                    : adaboost_params['learning_rate'].dropna().values.tolist()
}

In [9]:
grid_search     = GridSearchCV(estimator = adaboost_classifier,
                               param_grid = parameter_grid,
                               scoring = 'accuracy',
                               cv = 10,
                               verbose = 3)
grid_search     = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy   = grid_search.best_score_

Fitting 10 folds for each of 120 candidates, totalling 1200 fits
[CV] algorithm=SAMME, base_estimator__max_depth=1.0, learning_rate=0.1, n_estimators=40 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  algorithm=SAMME, base_estimator__max_depth=1.0, learning_rate=0.1, n_estimators=40, score=0.972, total=   0.2s
[CV] algorithm=SAMME, base_estimator__max_depth=1.0, learning_rate=0.1, n_estimators=40 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  algorithm=SAMME, base_estimator__max_depth=1.0, learning_rate=0.1, n_estimators=40, score=0.939, total=   0.2s
[CV] algorithm=SAMME, base_estimator__max_depth=1.0, learning_rate=0.1, n_estimators=40 


...

[CV]  algorithm=SAMME.R, base_estimator__max_depth=3.0, learning_rate=1.0, n_estimators=60, score=0.977, total=   0.8s
[CV] algorithm=SAMME.R, base_estimator__max_depth=3.0, learning_rate=1.0, n_estimators=60 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=3.0, learning_rate=1.0, n_estimators=60, score=0.986, total=   0.8s
[CV] algorithm=SAMME.R, base_estimator__max_depth=3.0, learning_rate=1.0, n_estimators=60 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=3.0, learning_rate=1.0, n_estimators=60, score=0.977, total=   0.8s
[CV] algorithm=SAMME.R, base_estimator__max_depth=3.0, learning_rate=1.0, n_estimators=60 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=3.0, learning_rate=1.0, n_estimators=60, score=0.972, total=   0.8s
[CV] algorithm=SAMME.R, base_estimator__max_depth=3.0, learning_rate=1.0, n_estimators=60 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=3.0, learning_rate=1.0, n_estimators=60, score=0.976, total=   0.8s
[CV] algorithm=SAMME.R, base_estimator__m

[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.5, n_estimators=40, score=0.986, total=   0.7s
[CV] algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.5, n_estimators=40 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.5, n_estimators=40, score=0.981, total=   0.7s
[CV] algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.5, n_estimators=40 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.5, n_estimators=40, score=0.972, total=   0.7s
[CV] algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.5, n_estimators=40 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.5, n_estimators=40, score=0.976, total=   0.7s
[CV] algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.5, n_estimators=40 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.5, n_estimators=40, score=0.986, total=   0.7s
[CV] algorithm=SAMME.R, base_estimator__m

[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.6, n_estimators=50, score=0.986, total=   0.9s
[CV] algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.6, n_estimators=50 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.6, n_estimators=50, score=0.977, total=   0.9s
[CV] algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.6, n_estimators=50 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.6, n_estimators=50, score=0.981, total=   0.9s
[CV] algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.6, n_estimators=50 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.6, n_estimators=50, score=0.972, total=   0.9s
[CV] algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.6, n_estimators=50 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.6, n_estimators=50, score=0.976, total=   0.9s
[CV] algorithm=SAMME.R, base_estimator__m

[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.9, n_estimators=60, score=0.995, total=   1.0s
[CV] algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.9, n_estimators=60 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.9, n_estimators=60, score=0.986, total=   1.0s
[CV] algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.9, n_estimators=60 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.9, n_estimators=60, score=0.986, total=   1.0s
[CV] algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.9, n_estimators=60 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.9, n_estimators=60, score=0.981, total=   1.0s
[CV] algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.9, n_estimators=60 
[CV]  algorithm=SAMME.R, base_estimator__max_depth=4.0, learning_rate=0.9, n_estimators=60, score=0.981, total=   1.0s
[CV] algorithm=SAMME.R, base_estimator__m

[Parallel(n_jobs=1)]: Done 1200 out of 1200 | elapsed: 11.3min finished


## Step 6 :- Analysing the best parameters of the model selected.
<div class="alert alert-block alert-info">
<b>Tip:</b> We find out the best parameters of the model as a result of Grid search implementation with accuracy being the performance measure considered here. 
</div>

In [50]:
print('The best parameters are :', best_parameters)

The best parameters are : {'algorithm': 'SAMME.R', 'base_estimator__max_depth': 4.0, 'learning_rate': 0.9, 'n_estimators': 60}


In [28]:
parameter_tune_results = pd.DataFrame(grid_search.cv_results_)
parameter_tune_results = parameter_tune_results.sort_values('mean_test_score',ascending=False)

In [49]:
parameter_tune_results.head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_base_estimator__max_depth,param_learning_rate,param_n_estimators,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
116,1.029348,0.006887,0.005585,0.000489,SAMME.R,4,0.9,60,"{'algorithm': 'SAMME.R', 'base_estimator__max_...",0.985915,...,0.985915,0.985915,0.981132,0.981132,0.981043,0.990521,0.985782,0.984920,0.005075,1
119,1.026655,0.013255,0.005884,0.000299,SAMME.R,4,1,60,"{'algorithm': 'SAMME.R', 'base_estimator__max_...",0.990610,...,0.990610,0.981221,0.981132,0.976415,0.981043,0.985782,0.985782,0.983506,0.006052,2
40,0.660832,0.005955,0.003292,0.000456,SAMME,3,0.9,50,"{'algorithm': 'SAMME', 'base_estimator__max_de...",0.985915,...,0.985915,0.981221,0.981132,0.976415,0.990521,0.990521,0.985782,0.983506,0.007039,2
56,1.029540,0.012024,0.004000,0.000019,SAMME,4,0.9,60,"{'algorithm': 'SAMME', 'base_estimator__max_de...",0.990610,...,0.981221,0.985915,0.981132,0.976415,0.981043,0.985782,0.990521,0.983506,0.004821,2
53,0.998331,0.008988,0.003988,0.000023,SAMME,4,0.6,60,"{'algorithm': 'SAMME', 'base_estimator__max_de...",0.990610,...,0.976526,0.981221,0.981132,0.985849,0.976303,0.985782,0.990521,0.983506,0.005667,2


## Step 7 :- Creating an instance of Adaboost classifier on basis of tuned parameters
<div class="alert alert-block alert-info">
<b>Final model to be used: </b> On the basis of parameter tuning, we create the model to be used in analysis.

In [51]:
decision_tree_classifier  = DecisionTreeClassifier(splitter     = 'best', 
                                                   criterion    = 'entropy',
                                                   max_depth    = best_parameters['base_estimator__max_depth'],
                                                   random_state = 11)
adaboost_classifier       = AdaBoostClassifier(base_estimator= decision_tree_classifier,
                                               algorithm     = best_parameters['algorithm'],
                                               learning_rate = best_parameters['learning_rate'],
                                               n_estimators  = best_parameters['n_estimators'],
                                               random_state  = 0)

In [52]:
adaboost_classifier.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='entropy',
                                                         max_depth=4.0,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=11,
                          

## Step 8 :- Predicting the output on test class

In [53]:
y_pred = adaboost_classifier.predict(X_test)

In [55]:
print('The accuracy attained by the model on the test set will be', accuracy_score(y_pred,y_test))

The accuracy attained by the model on the test set will be 0.9789674952198852


## Step 9 :- Performance measures 

In [56]:
print('The necessary confusion matrix is given by :- ')
confusion_matrix(y_pred,y_test)

The necessary confusion matrix is given by :- 


array([[491,  14],
       [  8, 533]], dtype=int64)

In [57]:
print('The classification report is given as :- ')
print(classification_report(y_pred,y_test))

The classification report is given as :- 
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       505
           1       0.97      0.99      0.98       541

    accuracy                           0.98      1046
   macro avg       0.98      0.98      0.98      1046
weighted avg       0.98      0.98      0.98      1046

